<a href="https://colab.research.google.com/github/cryptopatience/Main-code/blob/main/Mag_7_darkpool.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
# 필요한 라이브러리 설치 및 임포트
# !pip install yfinance pandas numpy plotly requests -q

import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import warnings
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio
import requests
from io import StringIO
import time

warnings.filterwarnings('ignore')
pio.renderers.default = 'colab'

# ==================== 설정 및 종목 리스트 ====================
MAG7_STOCKS = {
    'AAPL': 'Apple', 'MSFT': 'Microsoft', 'GOOGL': 'Alphabet',
    'AMZN': 'Amazon', 'NVDA': 'NVIDIA', 'META': 'Meta',
    'TSLA': 'Tesla', 'COIN': 'Coinbase', 'IBIT': 'Bitcoin ETF'
}

# ==================== 데이터 수집 함수 ====================

def get_market_volume(ticker, days_back=65):
    """Yahoo Finance에서 전체 시장 거래량 가져오기"""
    try:
        stock = yf.Ticker(ticker)
        end_date = datetime.now()
        start_date = end_date - timedelta(days=days_back + 10)
        df = stock.history(start=start_date, end=end_date)
        return df['Volume']
    except:
        return None

def get_finra_data_full(ticker, days_back=60):
    """
    FINRA 데이터 수집 및 핵심 지표 3가지 계산 (60일 history 포함)
    """
    try:
        today = datetime.now()
        data_list = []

        # Yahoo 전체 거래량 (분모용)
        market_volumes = get_market_volume(ticker, days_back)
        if market_volumes is None or market_volumes.empty: return None

        # FINRA 데이터 루프 (최근 N일)
        for days in range(days_back + 5): # 휴일 고려 여유분
            check_date = today - timedelta(days=days)
            if check_date.weekday() >= 5: continue

            date_str = check_date.strftime('%Y%m%d')
            date_key = check_date.strftime('%Y-%m-%d')

            url = f"https://cdn.finra.org/equity/regsho/daily/CNMSshvol{date_str}.txt"

            try:
                response = requests.get(url, timeout=3) # 타임아웃 짧게
                if response.status_code == 200:
                    df = pd.read_csv(StringIO(response.text), sep='|')
                    if 'Symbol' in df.columns: df.rename(columns={'Symbol': 'symbol'}, inplace=True)
                    if 'ShortVolume' in df.columns: df.rename(columns={'ShortVolume': 'shortVolume'}, inplace=True)
                    if 'TotalVolume' in df.columns: df.rename(columns={'TotalVolume': 'totalVolume'}, inplace=True)

                    row = df[df['symbol'] == ticker.upper()]

                    if not row.empty:
                        finra_total = int(row.iloc[0]['totalVolume']) # Dark Pool Vol
                        finra_short = int(row.iloc[0]['shortVolume']) # Dark Pool Short Vol

                        # Yahoo Volume 매칭
                        market_vol = 0
                        if date_key in market_volumes.index:
                            market_vol = market_volumes.loc[date_key]
                        else:
                            for idx in market_volumes.index:
                                if idx.strftime('%Y-%m-%d') == date_key:
                                    market_vol = market_volumes[idx]
                                    break

                        if market_vol > 0:
                            # 지표 계산
                            dp_ratio = (finra_total / market_vol) * 100
                            dp_short_ratio = (finra_short / finra_total) * 100
                            total_short_ratio = (finra_short / market_vol) * 100

                            # 보정 (데이터 오차)
                            if dp_ratio > 100: dp_ratio = 100

                            data_list.append({
                                'date': date_key,
                                'dp_ratio': round(dp_ratio, 2),
                                'dp_short_ratio': round(dp_short_ratio, 2),
                                'total_short_ratio': round(total_short_ratio, 2),
                                'market_vol': market_vol
                            })

                if len(data_list) >= days_back: break
            except:
                continue

        if not data_list: return None

        df_hist = pd.DataFrame(data_list).sort_values('date')

        # 10일 평균 및 변화율 계산 (통합 테이블용)
        df_hist['dp_short_ratio_10d_avg'] = df_hist['dp_short_ratio'].rolling(window=10, min_periods=1).mean()
        latest = df_hist.iloc[-1]

        recent_10d_avg = df_hist.iloc[-10:]['dp_short_ratio'].mean()
        dp_short_change = latest['dp_short_ratio'] - recent_10d_avg # 최근 1일 vs 10일 평균 차이

        return {
            'ticker': ticker,
            'name': MAG7_STOCKS[ticker],
            'latest_date': latest['date'],
            'dp_ratio': latest['dp_ratio'],
            'dp_short_ratio': latest['dp_short_ratio'],
            'total_short_ratio': latest['total_short_ratio'],
            'dp_short_10d_avg': latest['dp_short_ratio_10d_avg'],
            'dp_short_change_pct': dp_short_change, # 급락/급등 감지용
            'history': df_hist
        }

    except Exception as e:
        # print(f"Err {ticker}: {e}")
        return None

# ==================== 메인 분석 실행 ====================

print("="*80)
print("🚀 MAG 7 + Bitcoin: Dark Pool & Short Interest 심층 분석 (60일 데이터)")
print("="*80)

analysis_results = []
print("데이터 수집 중...", end=" ")

for ticker in MAG7_STOCKS.keys():
    print(f"{ticker}..", end=" ")
    res = get_finra_data_full(ticker, days_back=60)
    if res:
        analysis_results.append(res)

print("\n완료!")
df_main = pd.DataFrame([{k:v for k,v in r.items() if k != 'history'} for r in analysis_results])

# 추가 분석 컬럼 계산
df_main['dp_vs_total'] = (df_main['dp_short_ratio'] - df_main['total_short_ratio']).round(2)

# 시그널 컬럼 생성
def create_signal(row):
    # 1. 공매도 청산 신호 (DP Short 급락)
    if row['dp_short_change_pct'] < -5:
        return '🟢 급락 (청산 신호)'
    # 2. 기관의 강한 약세 포지션 (비공개 공매도 집중)
    if row['dp_ratio'] > 50 and row['dp_short_ratio'] > 55:
        return '🔴 기관 강한 약세 포지션'
    # 3. 기관 매집 가능성 (장외 매수 거래)
    if row['dp_ratio'] > 50 and row['dp_short_ratio'] < 45:
        return '💚 기관 매집 가능성'
    # 4. 비밀 공매도 집중 (DP_vs_Total)
    if row['dp_vs_total'] > 5:
        return '⚠️ DP에 공매도 집중'
    # 5. 투명한 거래 (Total_vs_DP)
    if row['dp_vs_total'] < -5:
        return '✅ 거래소에 공매도 집중'
    return '⚪ 관망/정상'

df_main['Signal'] = df_main.apply(create_signal, axis=1)
df_main = df_main.sort_values('dp_ratio', ascending=False)


# ==================== 0. 통합 상세 비교 테이블 출력 ====================

print("\n" + "="*100)
print("📚 Dark Pool 주요 신호 해석 가이드")
print("="*100)
print("```")
print("[DP_Ratio 높음 (>50%) + DP_Short 높음 (>55%)] → 🔴 기관의 강한 약세 포지션 (비공개 공매도 집중)")
print("[DP_Ratio 높음 (>50%) + DP_Short 낮음 (<45%)] → 💚 기관 매집 가능성 (장외 매수 거래)")
print("[DP_Short 급락 (10일 평균 대비 -5%p 이상)] → 🟢 공매도 청산! 상승 전환점")
print("[DP_Short > Total_Short (+5%p 초과)] → ⚠️ Dark Pool에 공매도 집중 (비밀 포지션)")
print("[Total_Short > DP_Short (+5%p 초과)] → ✅ 거래소에 공매도 집중 (투명한 거래)")
print("```")
print("\n" + "="*100)
print("📋 통합 상세 비교 테이블: Dark Pool & Short Interest 분석")
print("="*100)

# 출력용 데이터프레임 정리 및 포맷팅
df_display = df_main.rename(columns={
    'dp_ratio': 'DP_Ratio_%',
    'dp_short_ratio': 'DP_Short_%',
    'dp_short_10d_avg': 'DP_10D_Avg',
    'dp_short_change_pct': 'DP_1D_vs_10D',
    'total_short_ratio': 'Total_Short_%',
    'dp_vs_total': 'DP_vs_Total'
})

table_cols = ['ticker', 'name', 'DP_Ratio_%', 'DP_Short_%', 'DP_10D_Avg',
              'DP_1D_vs_10D', 'Total_Short_%', 'DP_vs_Total', 'Signal']

# 포맷팅 적용
df_display['DP_Ratio_%'] = df_display['DP_Ratio_%'].round(2).astype(str) + '%'
df_display['DP_Short_%'] = df_display['DP_Short_%'].round(2).astype(str) + '%'
df_display['DP_10D_Avg'] = df_display['DP_10D_Avg'].round(2).astype(str) + '%'
df_display['Total_Short_%'] = df_display['Total_Short_%'].round(2).astype(str) + '%'
df_display['DP_vs_Total'] = df_display['DP_vs_Total'].apply(lambda x: f"{x:+.2f}%p")
df_display['DP_1D_vs_10D'] = df_display['DP_1D_vs_10D'].apply(lambda x: f"{x:+.2f}%p")


print(df_display[table_cols].to_string(index=False))
print("="*100)


# --- 그래프 출력 섹션 (기존 코드 유지) ---

# ==================== 1. Dark Pool Ratio (단독 막대) ====================

print("\n📊 Chart 1: Dark Pool Ratio (기관의 장외 거래 장악력)")

fig1 = go.Figure()
colors_dp = ['green' if x < 40 else 'orange' if x < 50 else 'red' for x in df_main['dp_ratio']]

fig1.add_trace(go.Bar(
    x=df_main['ticker'], y=df_main['dp_ratio'],
    text=[f"{x}%" for x in df_main['dp_ratio']], textposition='auto',
    marker_color=colors_dp, name='Dark Pool %'
))

fig1.add_hline(y=50, line_dash="dash", line_color="red", annotation_text="기관 과열 (50%)")
fig1.add_hline(y=40, line_dash="dash", line_color="green", annotation_text="정상 범위 (40%)")

fig1.update_layout(title='Dark Pool Ratio (FINRA Vol / Total Vol)', height=400, template='plotly_white')
fig1.show()

# ---
# ==================== 2. Dark Pool Short vs Total Short (비교) ====================


print("\n📊 Chart 2: 은밀한 공매도(장외) vs 공개된 공매도 비교")

fig2 = go.Figure()

# Dark Pool 내부 공매도 비율
fig2.add_trace(go.Bar(
    x=df_main['ticker'], y=df_main['dp_short_ratio'],
    name='Dark Pool Short % (장외 내부)',
    marker_color='darkblue'
))

# 전체 거래 대비 공매도 비율
fig2.add_trace(go.Bar(
    x=df_main['ticker'], y=df_main['total_short_ratio'],
    name='Total Short % (전체 시장)',
    marker_color='gray'
))

fig2.update_layout(
    title='Dark Pool Short Ratio vs Total Market Short Ratio',
    barmode='group', height=400, template='plotly_white',
    yaxis_title='Short Ratio (%)'
)
fig2.show()

# ---
# ==================== 3. 상관관계 산점도 (버블) ====================


print("\n📊 Chart 3: Market Sentiment Map (4분면 분석)")

fig3 = go.Figure()

fig3.add_trace(go.Scatter(
    x=df_main['dp_ratio'],
    y=df_main['dp_short_ratio'],
    mode='markers+text',
    text=df_main['ticker'],
    textposition='top center',
    marker=dict(
        size=df_main['dp_ratio'] * 0.8, # 크기는 DP Ratio 비례
        color=df_main['dp_short_ratio'], # 색상은 Short Ratio 비례
        colorscale='RdYlGn_r', # 빨간색일수록 Short가 높음 (Bearish)
        showscale=True,
        colorbar=dict(title="Short Ratio")
    ),
    hovertemplate='<b>%{text}</b><br>DP Ratio: %{x}%<br>DP Short Ratio: %{y}%<extra></extra>'
))

# 4분면 기준선
fig3.add_vline(x=45, line_dash="dot", line_color="gray") # DP 관심도 기준
fig3.add_hline(y=50, line_dash="dot", line_color="gray") # Short 심리 기준

# 4분면 라벨
fig3.add_annotation(x=55, y=30, text="매집 (Accumulation)", showarrow=False, font=dict(color="green"))
fig3.add_annotation(x=55, y=70, text="매도/공매도 (Distribution)", showarrow=False, font=dict(color="red"))
fig3.add_annotation(x=35, y=30, text="개인/관망", showarrow=False, font=dict(color="gray"))

fig3.update_layout(
    title='X축: 기관 관심도 (DP Ratio) vs Y축: 공매도 심리 (DP Short Ratio)',
    xaxis_title='Dark Pool Ratio (%) - 높을수록 기관 개입 강함',
    yaxis_title='Dark Pool Short Ratio (%) - 높을수록 하락 베팅',
    height=500, template='plotly_white'
)
fig3.show()

# ---
# ==================== 4 & 5. 시계열 분석 (모든 종목 출력) ====================

print("\n" + "="*80)
print("📊 Chart 4 & 5: 전체 종목 상세 시계열 분석 (Dark Pool & Short Ratio)")
print("="*80)

# 분석된 모든 종목에 대해 반복 실행
for item in analysis_results:
    ticker = item['ticker']
    name = item['name']
    df_hist = item['history']

    # 현재 Dark Pool 비율 확인
    current_dp = item['dp_ratio']

    print(f"\n🔍 {ticker} ({name}) 분석 차트 생성 중... (현재 DP: {current_dp}%)")

    fig_ts = make_subplots(rows=2, cols=1, shared_xaxes=True,
                           subplot_titles=(f"{ticker} - Dark Pool Ratio Trend (기관 개입 강도)",
                                           f"{ticker} - Dark Pool Short Ratio Trend (공매도 심리)"),
                           vertical_spacing=0.15)

    # 시계열 1: Dark Pool Ratio (파란선)
    fig_ts.add_trace(go.Scatter(
        x=df_hist['date'], y=df_hist['dp_ratio'],
        mode='lines+markers', name='DP Ratio', line=dict(color='blue', width=2),
        showlegend=False
    ), row=1, col=1)

    # 과열 기준선 (50%)
    fig_ts.add_hline(y=50, line_dash="dot", line_color="red", annotation_text="과열 (50%)", row=1, col=1)
    fig_ts.add_hline(y=40, line_dash="dot", line_color="green", annotation_text="안정 (40%)", row=1, col=1)

    # 시계열 2: Dark Pool Short Ratio (주황선)
    fig_ts.add_trace(go.Scatter(
        x=df_hist['date'], y=df_hist['dp_short_ratio'],
        mode='lines+markers', name='DP Short Ratio', line=dict(color='orange', width=2),
        showlegend=False
    ), row=2, col=1)

    # 10일 평균선 추가 (시계열 2)
    fig_ts.add_trace(go.Scatter(
        x=df_hist['date'], y=df_hist['dp_short_ratio_10d_avg'],
        mode='lines', name='10일 평균', line=dict(color='gray', dash='dot'),
        showlegend=False
    ), row=2, col=1)


    # 매수/매도 분기점 (50%)
    fig_ts.add_hline(y=50, line_dash="dot", line_color="gray", annotation_text="매수/매도 분기점", row=2, col=1)

    fig_ts.update_layout(height=600, title_text=f"📊 {ticker} 60일 상세 타임라인", template='plotly_white')
    fig_ts.show()

🚀 MAG 7 + Bitcoin: Dark Pool & Short Interest 심층 분석 (60일 데이터)
데이터 수집 중... AAPL.. MSFT.. GOOGL.. AMZN.. NVDA.. META.. TSLA.. COIN.. IBIT.. 
완료!

📚 Dark Pool 주요 신호 해석 가이드
```
[DP_Ratio 높음 (>50%) + DP_Short 높음 (>55%)] → 🔴 기관의 강한 약세 포지션 (비공개 공매도 집중)
[DP_Ratio 높음 (>50%) + DP_Short 낮음 (<45%)] → 💚 기관 매집 가능성 (장외 매수 거래)
[DP_Short 급락 (10일 평균 대비 -5%p 이상)] → 🟢 공매도 청산! 상승 전환점
[DP_Short > Total_Short (+5%p 초과)] → ⚠️ Dark Pool에 공매도 집중 (비밀 포지션)
[Total_Short > DP_Short (+5%p 초과)] → ✅ 거래소에 공매도 집중 (투명한 거래)
```

📋 통합 상세 비교 테이블: Dark Pool & Short Interest 분석
ticker        name DP_Ratio_% DP_Short_% DP_10D_Avg DP_1D_vs_10D Total_Short_% DP_vs_Total        Signal
  TSLA       Tesla     57.85%     48.96%     48.93%      +0.02%p        28.32%    +20.64%p ⚠️ DP에 공매도 집중
  NVDA      NVIDIA     50.65%     39.72%     41.59%      -1.87%p        20.12%    +19.60%p   💚 기관 매집 가능성
  COIN    Coinbase     43.87%     58.68%     50.24%      +8.44%p        25.74%    +32.94%p ⚠️ DP에 공매도 집중
 GOOGL    Alphabet     42.83%     32


📊 Chart 2: 은밀한 공매도(장외) vs 공개된 공매도 비교



📊 Chart 3: Market Sentiment Map (4분면 분석)



📊 Chart 4 & 5: 전체 종목 상세 시계열 분석 (Dark Pool & Short Ratio)

🔍 AAPL (Apple) 분석 차트 생성 중... (현재 DP: 41.9%)



🔍 MSFT (Microsoft) 분석 차트 생성 중... (현재 DP: 37.98%)


...


🔍 NVDA (NVIDIA) 분석 차트 생성 중... (현재 DP: 50.65%)



🔍 META (Meta) 분석 차트 생성 중... (현재 DP: 38.83%)



🔍 TSLA (Tesla) 분석 차트 생성 중... (현재 DP: 57.85%)



🔍 COIN (Coinbase) 분석 차트 생성 중... (현재 DP: 43.87%)



🔍 IBIT (Bitcoin ETF) 분석 차트 생성 중... (현재 DP: 28.08%)
